In [2]:
from typing import Dict, List, Tuple, Callable, Generator, Any
import util
import sys
import logic
import game

from logic import conjoin, disjoin
from logic import PropSymbolExpr, Expr, to_cnf, pycoSAT, parseExpr, pl_true

import itertools
import copy

pacman_str = 'P'
food_str = 'FOOD'
wall_str = 'WALL'
pacman_wall_str = pacman_str + wall_str
DIRECTIONS = ['North', 'South', 'East', 'West']
blocked_str_map = dict([(direction, (direction + "_blocked").upper()) for direction in DIRECTIONS])
geq_num_adj_wall_str_map = dict([(num, "GEQ_{}_adj_walls".format(num)) for num in range(1, 4)])
DIR_TO_DXDY_MAP = {'North':(0, 1), 'South':(0, -1), 'East':(1, 0), 'West':(-1, 0)}

In [3]:
def sentence1() -> Expr:
    """Returns a Expr instance that encodes that the following expressions are all true.
    
    A or B
    (not A) if and only if ((not B) or C)
    (not A) or (not B) or C
    """
    "*** BEGIN YOUR CODE HERE ***"
    A = Expr('A')
    B = Expr('B')
    C = Expr('C')
    sentence1_1 = A | B
    sentence1_2 = (~A % (~B | C))
    sentence1_3 = disjoin(~A, ~B, C)
    return conjoin([sentence1_1, sentence1_2, sentence1_3])
    util.raiseNotDefined()
    "*** END YOUR CODE HERE ***"


def sentence2() -> Expr:
    """Returns a Expr instance that encodes that the following expressions are all true.
    
    C if and only if (B or D)
    A implies ((not B) and (not D))
    (not (B and (not C))) implies A
    (not D) implies C
    """
    "*** BEGIN YOUR CODE HERE ***"
    A = PropSymbolExpr('A')
    B = PropSymbolExpr('B')
    C = PropSymbolExpr('C')
    D = PropSymbolExpr('D')
    sentence2_1 = C % (B | D)
    sentence2_2 = A >> (~B & ~D)
    sentence2_3 = (~(B & ~C)) >> A
    sentence2_4 = ~D >> C
    return conjoin([sentence2_1, sentence2_2, sentence2_3, sentence2_4])
    util.raiseNotDefined()
    "*** END YOUR CODE HERE ***"


def sentence3() -> Expr:
    """Using the symbols PacmanAlive_1 PacmanAlive_0, PacmanBorn_0, and PacmanKilled_0,
    created using the PropSymbolExpr constructor, return a PropSymbolExpr
    instance that encodes the following English sentences (in this order):

    Pacman is alive at time 1 if and only if Pacman was alive at time 0 and it was
    not killed at time 0 or it was not alive at time 0 and it was born at time 0.

    Pacman cannot both be alive at time 0 and be born at time 0.

    Pacman is born at time 0.
    """
    "*** BEGIN YOUR CODE HERE ***"
    # A = Expr('PacmanAlive_1')
    # B = Expr('PacmanAlive_0')
    # C = Expr('PacmanBorn_0')
    # D = Expr('PacmanKilled_0')
    
    A = Expr('PacmanAlive_0')
    B = Expr('PacmanAlive_1')
    C = Expr('PacmanBorn_0')
    D = Expr('PacmanKilled_0')
    sentence3_1 = B % ((A & ~D) | (~A & C)) 
    sentence3_2 = ~(A & C)
    sentence3_3 = C
    return conjoin([sentence3_1, sentence3_2, sentence3_3])

    # util.raiseNotDefined()
    "*** END YOUR CODE HERE ***"

def findModel(sentence: Expr) -> Dict[Expr, bool]:
    """Given a propositional logic sentence (i.e. a Expr instance), returns a satisfying
    model if one exists. Otherwise, returns False.
    """
    cnf_sentence = to_cnf(sentence)
    # print(pycoSAT(cnf_sentence))
    return pycoSAT(cnf_sentence)

def findModelUnderstandingCheck() -> Dict[Expr, bool]:
    """Returns the result of findModel(Expr('a')) if lower cased expressions were allowed.
    You should not use findModel or Expr in this method.
    """
    a = Expr('A')
    "*** BEGIN YOUR CODE HERE ***"
    print("a.__dict__ is:", a.__dict__) # might be helpful for getting ideas
    class SimulatedfindModel:
        def __init__(self, variable_name: str = 'A'):
            self.set_variable_name(variable_name)
            
        def set_variable_name(self, value):
            self.variable_name = value
            
        def __repr__(self):
            return self.variable_name
    if a.__dict__['op'] == 'A':
        return {SimulatedfindModel('a'): True}
    else:
        return False

    util.raiseNotDefined()
    "*** END YOUR CODE HERE ***"

def entails(premise: Expr, conclusion: Expr) -> bool:
    """Returns True if the premise entails the conclusion and False otherwise.
    """
    "*** BEGIN YOUR CODE HERE ***"
    # Returns True if the premise entails the conclusion and False otherwise.
    entails = premise & ~(conclusion)
    if(findModel(entails) == False):
        return True
    else:
        return False
    util.raiseNotDefined()
    "*** END YOUR CODE HERE ***"

def plTrueInverse(assignments: Dict[Expr, bool], inverse_statement: Expr) -> bool:
    """Returns True if the (not inverse_statement) is True given assignments and False otherwise.
    pl_true may be useful here; see logic.py for its description.
    """
    "*** BEGIN YOUR CODE HERE ***"
    return pl_true(~inverse_statement, assignments)
    util.raiseNotDefined()
    "*** END YOUR CODE HERE ***"

In [4]:
print(findModel(sentence1()))
print(findModel(sentence2()))
print(findModel(sentence3()))

{B: True, A: False, C: True}
False
{PacmanAlive_0: False, PacmanKilled_0: False, PacmanAlive_1: True, PacmanBorn_0: True}


In [5]:
a = Expr('A')

In [6]:
findModelUnderstandingCheck()

a.__dict__ is: {'op': 'A', 'args': ()}


{a: True}

In [7]:
# QUESTION 3

def pacmanSuccessorAxiomSingle(x: int, y: int, time: int, walls_grid: List[List[bool]]=None) -> Expr:
    """
    Successor state axiom for state (x,y,t) (from t-1), given the board (as a 
    grid representing the wall locations).
    Current <==> (previous position at time t-1) & (took action to move to x, y)
    Available actions are ['North', 'East', 'South', 'West']
    Note that STOP is not an available action.
    """
    now, last = time, time - 1
    possible_causes: List[Expr] = [] # enumerate all possible causes for P[x,y]_t
    if walls_grid[x][y+1] != 1:
        possible_causes.append( PropSymbolExpr(pacman_str, x, y+1, time=last)
                            & PropSymbolExpr('South', time=last)) 
    if walls_grid[x][y-1] != 1:
        possible_causes.append( PropSymbolExpr(pacman_str, x, y-1, time=last) 
                            & PropSymbolExpr('North', time=last))
    if walls_grid[x+1][y] != 1:
        possible_causes.append( PropSymbolExpr(pacman_str, x+1, y, time=last) 
                            & PropSymbolExpr('West', time=last))
    if walls_grid[x-1][y] != 1:
        possible_causes.append( PropSymbolExpr(pacman_str, x-1, y, time=last) 
                            & PropSymbolExpr('East', time=last))
    if not possible_causes:
        return None
    # print("Possible Causes: ", possible_causes)
    "*** BEGIN YOUR CODE HERE ***"
    # print(PropSymbolExpr(pacman_str, x, y, time=now)) # P[x,y]_t, current position
    # print(disjoin(possible_causes)) # all possible causes for P[x,y]_t
    # print("Return: ", PropSymbolExpr(pacman_str, x, y, time=now) % disjoin(possible_causes)) # biconditional checking
    ## 檢查現在位置和所有可能的原因之間是否具有雙向關係(biconditional equivalence)
    return PropSymbolExpr(pacman_str, x, y, time=now) % disjoin(possible_causes) # disjoin : OR, conjoin : AND, % : biconditional
    util.raiseNotDefined()
    "*** END YOUR CODE HERE ***"


def SLAMSuccessorAxiomSingle(x: int, y: int, time: int, walls_grid: List[List[bool]]) -> Expr:
    """
    Similar to `pacmanSuccessorStateAxioms` but accounts for illegal actions
    where the pacman might not move timestep to timestep.
    Available actions are ['North', 'East', 'South', 'West']
    """
    now, last = time, time - 1
    moved_causes: List[Expr] = [] # enumerate all possible causes for P[x,y]_t, assuming moved to having moved
    if walls_grid[x][y+1] != 1:
        moved_causes.append( PropSymbolExpr(pacman_str, x, y+1, time=last)
                            & PropSymbolExpr('South', time=last))
    if walls_grid[x][y-1] != 1:
        moved_causes.append( PropSymbolExpr(pacman_str, x, y-1, time=last) 
                            & PropSymbolExpr('North', time=last))
    if walls_grid[x+1][y] != 1:
        moved_causes.append( PropSymbolExpr(pacman_str, x+1, y, time=last) 
                            & PropSymbolExpr('West', time=last))
    if walls_grid[x-1][y] != 1:
        moved_causes.append( PropSymbolExpr(pacman_str, x-1, y, time=last) 
                            & PropSymbolExpr('East', time=last))
    if not moved_causes:
        return None

    # ~P[x,y]_t-1 & ~W[x,y] & (P[x,y+1]_t-1 & South_t-1 | ...)
    moved_causes_sent: Expr = conjoin([~PropSymbolExpr(pacman_str, x, y, time=last) , ~PropSymbolExpr(wall_str, x, y), disjoin(moved_causes)]) 
    # print("Moved Causes Sent: ", moved_causes_sent)
    
    failed_move_causes: List[Expr] = [] # using merged variables, improves speed significantly
    auxilary_expression_definitions: List[Expr] = []
    for direction in DIRECTIONS:
        dx, dy = DIR_TO_DXDY_MAP[direction]
        print(dx, dy)
        wall_dir_clause = PropSymbolExpr(wall_str, x + dx, y + dy) & PropSymbolExpr(direction, time=last)
        wall_dir_combined_literal = PropSymbolExpr(wall_str + direction, x + dx, y + dy, time=last)
        failed_move_causes.append(wall_dir_combined_literal)
        auxilary_expression_definitions.append(wall_dir_combined_literal % wall_dir_clause)

    failed_move_causes_sent: Expr = conjoin([
        PropSymbolExpr(pacman_str, x, y, time=last),
        disjoin(failed_move_causes)])

    return conjoin([PropSymbolExpr(pacman_str, x, y, time=now) % disjoin([moved_causes_sent, failed_move_causes_sent])] + auxilary_expression_definitions)


def pacphysicsAxioms(t: int, all_coords: List[Tuple], non_outer_wall_coords: List[Tuple], walls_grid: List[List] = None, sensorModel: Callable = None, successorAxioms: Callable = None) -> Expr:
    """
    Given:
        t: timestep
        all_coords: list of (x, y) coordinates of the entire problem
        non_outer_wall_coords: list of (x, y) coordinates of the entire problem,
            excluding the outer border (these are the actual squares pacman can
            possibly be in)
        walls_grid: 2D array of either -1/0/1 or T/F. Used only for successorAxioms.
            Do NOT use this when making possible locations for pacman to be in.
        sensorModel(t, non_outer_wall_coords) -> Expr: function that generates
            the sensor model axioms. If None, it's not provided, so shouldn't be run.
        successorAxioms(t, walls_grid, non_outer_wall_coords) -> Expr: function that generates
            the sensor model axioms. If None, it's not provided, so shouldn't be run.
    Return a logic sentence containing all of the following:
        - for all (x, y) in all_coords:
            If a wall is at (x, y) --> Pacman is not at (x, y)
        - Pacman is at exactly one of the squares at timestep t.
        - Pacman takes exactly one action at timestep t.
        - Results of calling sensorModel(...), unless None.
        - Results of calling successorAxioms(...), describing how Pacman can end in various
            locations on this time step. Consider edge cases. Don't call if None.
    """
    pacphysics_sentences = []

    "*** BEGIN YOUR CODE HERE ***"
    possiblePositions = []  
    for (x,y) in non_outer_wall_coords:
        print("non_outer_wall_coords", PropSymbolExpr(pacman_str, x, y, time = t))
        possiblePositions.append(PropSymbolExpr(pacman_str, x, y, time = t)) # all the possible positions that pacman can be at time = t
    # - for all (x, y) in all_coords:
    #         If a wall is at (x, y) --> Pacman is not at (x, y)
    for (x,y) in all_coords:
        # Why not using biconditional here? because we only need to check if there is a wall at (x,y) then pacman is not at (x,y)
        # If there is no wall at (x,y), we don't need to check if pacman is at (x,y) then there is no wall at (x,y)
        print("all_coords", PropSymbolExpr(wall_str, x, y) >> ~(PropSymbolExpr(pacman_str, x, y, time=t)))
        pacphysics_sentences.append(PropSymbolExpr(wall_str, x, y) >> ~(PropSymbolExpr(pacman_str, x, y, time=t)))   
    
    # - Pacman is at exactly one of the squares at timestep t.
    pacphysics_sentences.append(exactlyOne(possiblePositions)) # Make sure pacman is at exactly one of the squares at timestep t
    
    possibleActions = []  
    for action in DIRECTIONS:
        print("PossibleActions:", PropSymbolExpr(action, time = t))
        possibleActions.append(PropSymbolExpr(action, time = t)) # all the possible actions that pacman can take at time = t
        
    # - Pacman takes exactly one action at timestep t.
    pacphysics_sentences.append(exactlyOne(possibleActions))
    
    # - Results of calling sensorModel(...), unless None.
    if sensorModel is not None:
        print("sensorModel", sensorModel(t, non_outer_wall_coords))
        pacphysics_sentences.append(sensorModel(t, non_outer_wall_coords= non_outer_wall_coords))
        
    # - Results of calling successorAxioms(...), describing how Pacman can end in various
    #         locations on this time step. Consider edge cases. Don't call if None.
    if t > 0 and successorAxioms is not None: # t > 0 because we already know where pacman is at t = 0
        print("successorAxioms", successorAxioms(t, walls_grid, non_outer_wall_coords))
        pacphysics_sentences.append(successorAxioms(t, walls_grid, non_outer_wall_coords))
        
    print(conjoin(pacphysics_sentences))
    return conjoin(pacphysics_sentences) # conjoin : AND, return all the sentences
    util.raiseNotDefined()
    "*** END YOUR CODE HERE ***"

    return conjoin(pacphysics_sentences)


def checkLocationSatisfiability(x1_y1: Tuple[int, int], x0_y0: Tuple[int, int], action0, action1, problem):
    """
    Given:
        - x1_y1 = (x1, y1), a potential location at time t = 1
        - x0_y0 = (x0, y0), Pacman's location at time t = 0
        - action0 = one of the four items in DIRECTIONS, Pacman's action at time t = 0
        - action1 = to ensure match with autograder solution
        - problem = an instance of logicAgents.LocMapProblem
    Note:
        - there's no sensorModel because we know everything about the world
        - the successorAxioms should be allLegalSuccessorAxioms where needed
    Return:
        - a model where Pacman is at (x1, y1) at time t = 1
        - a model where Pacman is not at (x1, y1) at time t = 1
    """
    walls_grid = problem.walls
    walls_list = walls_grid.asList()
    all_coords = list(itertools.product(range(problem.getWidth()+2), range(problem.getHeight()+2)))
    non_outer_wall_coords = list(itertools.product(range(1, problem.getWidth()+1), range(1, problem.getHeight()+1)))
    KB = []
    x0, y0 = x0_y0
    x1, y1 = x1_y1

    # We know which coords are walls:
    map_sent = [PropSymbolExpr(wall_str, x, y) for x, y in walls_list]
    KB.append(conjoin(map_sent)) # 所有 wall 的位置

    "*** BEGIN YOUR CODE HERE ***"
    # 所有 t = 0 時可能的位置(sentences)
    KB.append(pacphysicsAxioms(0, all_coords, non_outer_wall_coords, walls_grid, sensorModel= None, successorAxioms=allLegalSuccessorAxioms))
    # 所有 t = 1 時可能的位置(sentences)
    KB.append(pacphysicsAxioms(1, all_coords, non_outer_wall_coords,walls_grid, sensorModel= None, successorAxioms= allLegalSuccessorAxioms))
    # Pacman 在 t = 0 時的位置
    KB.append(PropSymbolExpr(pacman_str, x0, y0, time = 0))
    # Pacman 在 t = 0 時的行動
    KB.append(PropSymbolExpr(action0, time = 0))
    # Pacman 在 t = 1 時的行動
    KB.append(PropSymbolExpr(action1, time=1))
    # where Pacman is at (x1, y1) at time t = 1, 找到能滿足所有條件所有座位的值
    # conjoin(KB) : 所有可能的位置（KB）， PropSymbolExpr(pacman_str, x1, y1, time=1)]) : Pacman 在 (x1, y1) 上
    model1 = findModel(conjoin([conjoin(KB), PropSymbolExpr(pacman_str, x1, y1, time=1)]))
    print("Model1: ", len(KB), model1)
    # where Pacman is not at (x1, y1) at time t = 1, 找到能滿足所有條件所有座位的值
    # conjoin(KB) : 所有可能的位置（KB）， ~PropSymbolExpr(pacman_str, x1, y1, time=1)]) : Pacman 不在 (x1, y1) 上
    model2 = findModel(conjoin([conjoin(KB), ~PropSymbolExpr(pacman_str, x1, y1, time = 1)]))
    print("Model2: ", len(KB), model2)
    return (model1, model2)
    util.raiseNotDefined()
    "*** END YOUR CODE HERE ***"